In [9]:
import pandas as pd

original_df = pd.read_csv("results/Original.csv", index_col=0)
eval_df = pd.read_csv("results/basic_QA.csv", index_col=0)

In [ ]:
import re, glob
references = list(map(lambda x: int(re.findall("([0-9]+).txt", x)[0]), glob.glob("reference_txts/*")))

original_df = original_df[original_df["References"].isin(references)]
# Fill compositions in original for matching
original_df["Composition"] = original_df["Composition"].fillna(method='ffill')

In [18]:
# Need to first fix the composition of eval_df to be in alphabetic order - or no exact matching can be done
def composition_permute(c):
    elements = re.findall("[A-Z][^A-Z]*", c)
    elements = sorted(elements)
    return "".join(elements)
eval_df["Composition"] = eval_df["Composition"].apply(composition_permute)

### Choosing to evaluate the "Composition" column using precision and recall. 

Precision = How many of the ones in eval are actually present?  
Recall = How many of the ones in original are in eval

In [79]:
# There are retrievals for 'x' (Alx) in eval that might correspond to 
# numbers in the original (Al2). So elaborate procedure for matching :(
composition_matching = {} 

for comp in eval_df["Composition"]:
    if 'x' in comp:
        # Do a regex matching
        composition_matching[comp] = original_df["Composition"][original_df["Composition"].apply(lambda x: len(re.findall(comp.replace("x", "[0-9\.]+"), x)) > 0)].values.tolist()
    else:
        if comp in original_df["Composition"].tolist():
            composition_matching[comp] = [comp]
        else:
            composition_matching[comp] = []


composition_precision =  sum([len(composition_matching[key])>0 for key in composition_matching]) / len(composition_matching.keys())
composition_recall =  len(sum(list(composition_matching.values()), [])) / original_df.shape[0]

In [80]:
composition_precision, composition_recall

(0.7317073170731707, 0.3333333333333333)

Recall is likely low because my pdf-to-txt method is not very good, so a lot of relevant information might have been missed from the txt files. Nothing much I can do about that, unless I get good data. Precision seems to be decent though, for a first attempt. 

Now the other columns should likely be evaluated on accuracy, but how to determine accuracy for a column with multiple values is task-dependent. I'll go with the simplest - if I got it even partly right, it counts.

In [82]:
original_df.columns

Index(['Alloy System', 'Composition', 'Processing Condition', 'SS or IM',
       'Phases', '# EL', '# Phases', 'Property', 'References'],
      dtype='object')

In [83]:
eval_df.columns

Index(['Composition', 'Processing Condition', 'SS or IM', 'Phases', 'Property',
       'References'],
      dtype='object')

In [94]:
from IPython.display import display

In [ ]:
columns = ["Processing Condition", "SS or IM", "Phases", "Property"]
correct = [0 for c in columns]
incorrect = [0 for c in columns]

for i in range(eval_df.shape[0]):
    comp = eval_df.iloc[i]["Composition"]
    ref = eval_df.iloc[i]["References"]
    if not composition_matching[comp]:
        continue
    corr_original_df = original_df[original_df["Composition"].isin(composition_matching[comp]) & (original_df["References"]==ref)]

    # For each row
    for j in range(corr_original_df.shape[0]):
        # Add to accuracy for each column
        for k, col in enumerate(columns):
            if not corr_original_df.iloc[j][col] or (corr_original_df.iloc[j][col] in ["unk", "Unk"]):
                continue
            if eval_df.iloc[i][col] in str(corr_original_df.iloc[j][col]):
                correct[k] += 1
            else:
                incorrect[k] += 1

In [105]:
print(correct, incorrect)

[71, 63, 47, 54] [5, 10, 29, 22]


In [113]:
# Per column accuracy
per_col_acc = {
    col: [correct[k]/(correct[k] + incorrect[k])] for k, col in enumerate(columns)
}

per_col_df = pd.DataFrame(per_col_acc)
print("Column-wise accuracy")
display(per_col_df)

print("Overall:", per_col_df.values.mean())

Column-wise accuracy


,Processing Condition,SS or IM,Phases,Property
0,0.934211,0.863014,0.618421,0.710526


Overall: 0.7815428983417447
